In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece 
!sudo apt-get install swig
!sudo pip install jamspell
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [2]:
import string
import pandas as pd
import numpy as np

import datetime as dt
from datetime import datetime


import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import jamspell
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import torch
from transformers import AutoConfig, XLNetTokenizer, XLNetModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, set_seed

from sklearn.metrics import classification_report
from scipy.special import softmax
from tqdm import tqdm

import datasets
from datasets import load_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
MODEL = f"xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
dataset = datasets.load_dataset('tweet_eval', 'sentiment')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')
lemmatizer= WordNetLemmatizer()
punct = ''.join([i for i in string.punctuation if i not in [',','?','!','.']])

def light_clean(x):
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos las palabras que empiezan por @.
  x = re.sub(r'@[\w]+', '', x)
  # Signos de puntuacion menos. 
  x = x.translate(str.maketrans('', '', punct))
  x = x.replace('#', '')
  # Spell check
  x = corrector.FixFragment(x)
  return x

In [6]:
# vectorizamos la funcion utilizada para limpiar el texto.
clean_vect = np.vectorize(light_clean)

In [7]:
# Aplicamos la funcion en la training, validation y test set.
data = dataset.map(lambda e: {"clean": clean_vect(e['text'])}, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
#tokenizamos 
data = data.map(lambda e: tokenizer(e['clean'], truncation=True, max_length=512), batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
# Sets en Formato Dict, datos listos para darselos al modelo.
train_dataset = data['train']
val_dataset = data['validation']
test_dataset = data['test']

In [10]:
train_df = dataset['train'].to_pandas()
test_df = data['test'].to_pandas()

In [11]:
class_weights = (1 - (train_df['label'].value_counts().sort_index() / len(train_df))).values
class_weights = torch.from_numpy(class_weights).float().to("cuda")

In [12]:
train_dataset = train_dataset.rename_column('label', 'labels')

In [13]:
from torch import nn

In [14]:
class WeightLossTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    # feed inputs to the model and extract logits
    outputs = model(**inputs)
    logits = outputs.get('logits')
    # extract label
    labels = inputs.get('labels')
    # define loss function with lass weights
    loss_func = nn.CrossEntropyLoss(weight=class_weights)
    # compute loss
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

In [15]:
modelFineTuned = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [16]:
batch_size = 32
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='./res',
    num_train_epochs=5, 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True
)

In [17]:
trainer = WeightLossTrainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer
                            )

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text, clean. If text, clean are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45615
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7130
  Number of trainable parameters = 117311235


Epoch,Training Loss,Validation Loss
1,0.708500,0.661109
2,0.554700,0.648268
3,0.438700,0.751155
4,0.339000,0.844723
5,0.273900,0.961143


The following columns in the evaluation set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text, clean. If text, clean are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to ./res/checkpoint-1426
Configuration saved in ./res/checkpoint-1426/config.json
Model weights saved in ./res/checkpoint-1426/pytorch_model.bin
tokenizer config file saved in ./res/checkpoint-1426/tokenizer_config.json
Special tokens file saved in ./res/checkpoint-1426/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text, clean. If text, clean are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch s

TrainOutput(global_step=7130, training_loss=0.4628072125380036, metrics={'train_runtime': 2256.0834, 'train_samples_per_second': 101.093, 'train_steps_per_second': 3.16, 'total_flos': 5595438949560180.0, 'train_loss': 0.4628072125380036, 'epoch': 5.0})

In [19]:
test_preds_raw, test_labels , _ = trainer.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

The following columns in the test set don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text, clean. If text, clean are not expected by `XLNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 12284
  Batch size = 32


              precision    recall  f1-score   support

           0      0.699     0.729     0.714      3972
           1      0.723     0.658     0.689      5937
           2      0.635     0.731     0.679      2375

    accuracy                          0.695     12284
   macro avg      0.686     0.706     0.694     12284
weighted avg      0.698     0.695     0.695     12284

